# The 4-way Intersection

`pyspect` can set up problems independently from implementations...
At this stage, we only need to import pyspect and select a pyspect language...

In [1]:
from math import pi

from pyspect import *
from pyspect.langs.ltl import *

TLT.select(ContinuousLTL)

In [2]:
x = EMPTY
print(x.__require__)
x = AppliedSet('complement', x)
print('x:', x.__require__)
y = EMPTY
t = And(x, y)
print(type(t), t.__require__)

('empty',)
x: ('complement', 'empty')
<class 'pyspect.tlt.TLT'> ('complement', 'intersect', 'empty')


## Environment

Let us first setup the 4-way intersection environment...

In [3]:
## CONSTANTS ##

# Define origin and size of area, makes it easier to scale up/down later on 
X0, XN = -1.2, 2.4
Y0, YN = -1.2, 2.4

In `pyspect`, we are able to define sets abstractly. They are evaluated [lazily](https://wikipedia.com). For ease-of-use we provide functions for this:
- `BoundedSet`
- `HalfSpaceSet`
- `...`

In [4]:
## TRAFFIC RULES ##

speedlimit = BoundedSet(v=(0.3, 0.6))

A benefit of using `pyspects` TLTs is that we can mix set definitions and logic statements...
To comply with `center`, a system needs to be member of all provided sets and other defined rules, e.g. `speedlimit`. 

In [5]:
## INTERSECTION CENTER ##

center = And(
    BoundedSet(x=(X0 + 0.2*XN, X0 + 0.8*XN), y=(Y0 + 0.2*YN, Y0 + 0.8*YN)),
    HalfSpaceSet(normal=[+1, +1], offset=[X0 + 0.25*XN, Y0 + 0.25*YN]),
    HalfSpaceSet(normal=[-1, +1], offset=[X0 + 0.75*XN, Y0 + 0.25*YN]),
    HalfSpaceSet(normal=[-1, -1], offset=[X0 + 0.75*XN, Y0 + 0.75*YN]),
    HalfSpaceSet(normal=[+1, -1], offset=[X0 + 0.25*XN, Y0 + 0.75*YN]),
    speedlimit
)

We see this here as well...

In [6]:
## ROADS ##

road_e = BoundedSet(y=(Y0 + 0.3*YN, Y0 + 0.5*YN), h=(-pi/5, +pi/5))
road_e = Or(And(road_e, speedlimit), center)

road_w = BoundedSet(y=(Y0 + 0.5*YN, Y0 + 0.7*YN), h=(+pi - pi/5, -pi + pi/5))
road_w = Or(And(road_w, speedlimit), center)

road_n = BoundedSet(x=(X0 + 0.5*XN, X0 + 0.7*XN), h=(+pi/2 - pi/5, +pi/2 + pi/5))
road_n = Or(And(road_n, speedlimit), center)

road_s = BoundedSet(x=(X0 + 0.3*XN, X0 + 0.5*XN), h=(-pi/2 - pi/5, -pi/2 + pi/5))
road_s = Or(And(road_s, speedlimit), center)

Later in this example, we will primarily work with entries and exits...

In [7]:
## ENTRIES ##

entry_e = And(BoundedSet(x=(X0 + 0.85*XN, X0 + 1.00*XN), y=(Y0 + 0.53*YN, Y0 + 0.67*YN)), road_w)
entry_w = And(BoundedSet(x=(X0 + 0.00*XN, X0 + 0.15*XN), y=(Y0 + 0.33*YN, Y0 + 0.47*YN)), road_e)
entry_n = And(BoundedSet(x=(X0 + 0.33*XN, X0 + 0.47*XN), y=(Y0 + 0.85*YN, Y0 + 1.00*YN)), road_s)
entry_s = And(BoundedSet(x=(X0 + 0.53*XN, X0 + 0.67*XN), y=(Y0 + 0.00*YN, Y0 + 0.15*YN)), road_n)


## EXITS ##

exit_e = And(BoundedSet(x=(X0 + 0.85*XN, X0 + 1.00*XN), y=(Y0 + 0.33*YN, Y0 + 0.47*YN)), road_e)
exit_w = And(BoundedSet(x=(X0 + 0.00*XN, X0 + 0.15*XN), y=(Y0 + 0.53*YN, Y0 + 0.67*YN)), road_w)
exit_n = And(BoundedSet(x=(X0 + 0.53*XN, X0 + 0.67*XN), y=(Y0 + 0.85*YN, Y0 + 1.00*YN)), road_n)
exit_s = And(BoundedSet(x=(X0 + 0.33*XN, X0 + 0.47*XN), y=(Y0 + 0.00*YN, Y0 + 0.15*YN)), road_s)

We can define new functions...

In [8]:
def FollowTo(constraint, goal_or_constraint, *more):
    if more: # 2nd arg is constraint
        return FollowTo(And(constraint, goal_or_constraint), *more)
    else: # 2nd arg is exit
        return Until(constraint, goal_or_constraint)

Three vehicles in the scenario must follow these rules...

In [9]:
vehicle_1 = FollowTo(Or(road_e, road_n), exit_n)

vehicle_2 = FollowTo(road_n, Not(vehicle_1), exit_n)

vehicle_3 = FollowTo(road_e, Not(vehicle_1), Not(vehicle_2), exit_e)

## Implementations

Implementations and set representations are completely general. To demonstrate, we define the simplest implementation.
`StrImpl` produces strings that "represent" the sets. This is completetly fine as well...  

In [10]:
class StrImpl:

    def __init__(self, ndim: int) -> None:
        self.ndim = ndim

    def set_axes_names(self, *args):
        assert len(args) == self.ndim
        self._axes_names = args

    def axis(self, name: str) -> int:
        return self._axes_names.index(name)

    def axis_name(self, i: int) -> str:
        return self._axes_names[i]
    
    def axis_is_periodic(self, i: int) -> bool:
        return False

    def plane_cut(self, normal, offset, axes=None):
        return 'plane{...}'

    def empty(self):
        return 'empty{ }'
    
    def complement(self, vf):
        return f'({vf})^C'
    
    def intersect(self, vf1, vf2):
        return f'({vf1} ∩ {vf2})'

    def union(self, vf1, vf2):
        return f'({vf1} ∪ {vf2})'
    
    def reach(self, target, constraints=None):
        return f'Reach({target}, {constraints})'

    def avoid(self, target, constraints=None):
        return f'Avoid({target}, {constraints})'

In [13]:
impl = StrImpl(ndim=4)
impl.set_axes_names('x', 'y', 'h', 'v')

out = TLT(vehicle_2).realize(impl)

print(out)

Reach(((((empty{ })^C ∩ (plane{...} ∩ plane{...})) ∩ (plane{...} ∩ plane{...})) ∩ (((((empty{ })^C ∩ (plane{...} ∩ plane{...})) ∩ (plane{...} ∩ plane{...})) ∩ ((empty{ })^C ∩ (plane{...} ∩ plane{...}))) ∪ ((((((((empty{ })^C ∩ (plane{...} ∩ plane{...})) ∩ (plane{...} ∩ plane{...})) ∩ plane{...}) ∩ plane{...}) ∩ plane{...}) ∩ plane{...}) ∩ ((empty{ })^C ∩ (plane{...} ∩ plane{...}))))), ((((((empty{ })^C ∩ (plane{...} ∩ plane{...})) ∩ (plane{...} ∩ plane{...})) ∩ ((empty{ })^C ∩ (plane{...} ∩ plane{...}))) ∪ ((((((((empty{ })^C ∩ (plane{...} ∩ plane{...})) ∩ (plane{...} ∩ plane{...})) ∩ plane{...}) ∩ plane{...}) ∩ plane{...}) ∩ plane{...}) ∩ ((empty{ })^C ∩ (plane{...} ∩ plane{...})))) ∩ (Reach(((((empty{ })^C ∩ (plane{...} ∩ plane{...})) ∩ (plane{...} ∩ plane{...})) ∩ (((((empty{ })^C ∩ (plane{...} ∩ plane{...})) ∩ (plane{...} ∩ plane{...})) ∩ ((empty{ })^C ∩ (plane{...} ∩ plane{...}))) ∪ ((((((((empty{ })^C ∩ (plane{...} ∩ plane{...})) ∩ (plane{...} ∩ plane{...})) ∩ plane{...}) ∩ plane

Implementation using `hj_reachability`.

In [14]:
import numpy as np
import hj_reachability as hj

class HJImpl:

    solver_settings = hj.SolverSettings.with_accuracy("low")

    def __init__(self, dynamics, grid, time_horizon):
        self.grid = grid
        self.ndim = grid.ndim
        self.dynamics = dynamics
        self.timeline = self.new_timeline(time_horizon)
           
    def new_timeline(self, target_time, start_time=0, time_step=0.2):
        assert time_step > 0
        is_forward = target_time >= start_time
        target_time += 1e-5 if is_forward else -1e-5
        time_step *= 1 if is_forward else -1
        return np.arange(start_time, target_time, time_step)

    def set_axes_names(self, *args):
        assert len(args) == self.ndim
        self._axes_names = tuple(args)

    def axis(self, name: str) -> int:
        assert name in self._axes_names, f'Axis ({name=}) does not exist.'
        return self._axes_names.index(name)

    def axis_name(self, i: int) -> str:
        assert i < len(self._axes_names), f'Axis ({i=}) does not exist.'
        return self._axes_names[i]

    def axis_is_periodic(self, i: int) -> bool:
        assert i < len(self._axes_names), f'Axis ({i=}) does not exist.'
        return bool(self.grid._is_periodic_dim[i])

    def plane_cut(self, normal, offset, axes=None):
        data = np.zeros(self.grid.shape)
        axes = axes or list(range(self.grid.ndim))
        x = lambda i: self.grid.states[..., i]
        for i, k, m in zip(axes, normal, offset):
            data -= k*x(i) - k*m
        return data

    def empty(self):
        return np.ones(self.grid.shape)*np.inf
    
    def complement(self, vf):
        return np.asarray(-vf)
    
    def intersect(self, vf1, vf2):
        return np.maximum(vf1, vf2)

    def union(self, vf1, vf2):
        return np.minimum(vf1, vf2)
    
    def reach(self, target, constraints=None):
        self.dynamics.with_mode('reach')
        if not self.is_invariant(target):
            target = np.flip(target, axis=0)
        if not self.is_invariant(constraints):
            constraints = np.flip(constraints, axis=0)
        vf = hj.solve(self.solver_settings,
                      self.dynamics,
                      self.grid,
                      -self.timeline,
                      target,
                      constraints)
        return np.flip(np.asarray(vf), axis=0)
    
    def avoid(self, target, constraints=None):
        self.dynamics.with_mode('avoid')
        if not self.is_invariant(target):
            target = np.flip(target, axis=0)
        if not self.is_invariant(constraints):
            constraints = np.flip(constraints, axis=0)
        vf = hj.solve(self.solver_settings,
                      self.dynamics,
                      self.grid,
                      -self.timeline,
                      target,
                      constraints)
        return np.flip(np.asarray(vf), axis=0)

    def project_onto(self, vf, *idxs, keepdims=False, union=True):
        idxs = [len(vf.shape) + i if i < 0 else i for i in idxs]
        dims = [i for i in range(len(vf.shape)) if i not in idxs]
        if union:
            return vf.min(axis=tuple(dims), keepdims=keepdims)
        else:
            return vf.max(axis=tuple(dims), keepdims=keepdims)

    def is_invariant(self, vf):
        return (True if vf is None else
                len(vf.shape) != len(self.timeline.shape + self.grid.shape))

    def make_tube(self, vf):
        return (vf if not self.is_invariant(vf) else
                np.concatenate([vf[np.newaxis, ...]] * len(self.timeline)))

/home/kaarmu/.pyenv/versions/3.12.7/envs/pyspect/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
from hj_reachability.systems import Bicycle4D
from pyspect.plotting.levelsets import *

reach_dynamics = Bicycle4D(min_steer=-5*pi/4,
                           max_steer=+5*pi/4,
                           min_accel=-0.4,
                           max_accel=+0.4)

min_bounds = np.array([   X0,    Y0, -pi, +0.3])
max_bounds = np.array([XN+X0, YN+Y0, +pi, +0.8])
grid = hj.Grid.from_lattice_parameters_and_boundary_conditions(hj.sets.Box(min_bounds, max_bounds),
                                                               (31, 31, 25, 7),
                                                               periodic_dims=2)

impl = HJImpl(reach_dynamics, grid, 3)
impl.set_axes_names('x', 'y', 'h', 'v')

out = TLT(vehicle_1).realize(impl)

plot3D_levelset(impl.project_onto(out, 0, 1, 2),
                min_bounds=[0, *min_bounds[:2]],
                max_bounds=[3, *max_bounds[:2]])

sb1=<ReferredSet at 0x000007f5bf0519a6>: ()
sb2=<ReferredSet at 0x000007f5bf051b17>: ()
x=<AppliedSet at 0x000007f5bf055e45>: ('intersect',)
sb1=<ReferredSet at 0x000007f5bf059125>: ()
sb2=<ReferredSet at 0x000007f5bf05912e>: ()
x=<AppliedSet at 0x000007f5bf059059>: ('intersect',)
sb1=<AppliedSet at 0x000007f5bf059059>: ('intersect',)
sb2=<ReferredSet at 0x000007f5bf05912b>: ()
x=<AppliedSet at 0x000007f5bf0590cb>: ('intersect',)
sb1=<AppliedSet at 0x000007f5bf0590cb>: ('intersect',)
sb2=<ReferredSet at 0x000007f5bf05913a>: ()
x=<AppliedSet at 0x000007f5bf05906b>: ('intersect',)
sb1=<AppliedSet at 0x000007f5bf05906b>: ('intersect',)
sb2=<ReferredSet at 0x000007f5bf059149>: ()
x=<AppliedSet at 0x000007f5bf053b32>: ('intersect',)
sb1=<AppliedSet at 0x000007f5bf053b32>: ('intersect',)
sb2=<ReferredSet at 0x000007f5bf059152>: ()
x=<AppliedSet at 0x000007f5bf051b08>: ('intersect',)
sb1=<ReferredSet at 0x000007f5bf05915b>: ()
sb2=<ReferredSet at 0x000007f5bf059173>: ()
x=<AppliedSet at 0x000

Running for 15 iterations: 100%|██████████| 15/15 [00:07<00:00,  2.14it/s]


In [16]:
from hj_reachability.systems import Bicycle4D
from pyspect.plotting.levelsets import *

reach_dynamics = Bicycle4D(min_steer=-5*pi/4,
                           max_steer=+5*pi/4,
                           min_accel=-0.4,
                           max_accel=+0.4)

min_bounds = np.array([   X0,    Y0, -pi, +0.3])
max_bounds = np.array([XN+X0, YN+Y0, +pi, +0.8])
grid = hj.Grid.from_lattice_parameters_and_boundary_conditions(hj.sets.Box(min_bounds, max_bounds),
                                                               (31, 31, 25, 7),
                                                               periodic_dims=2)

impl = HJImpl(reach_dynamics, grid, 3)
impl.set_axes_names('x', 'y', 'h', 'v')

rect = BoundedSet(x=(X0 + 0.2*XN, X0 + 0.8*XN), y=(Y0 + 0.2*YN, Y0 + 0.8*YN))
wall = HalfSpaceSet(normal=[+1, +1], offset=[X0 + 0.25*XN, Y0 + 0.75*YN])

region = And(rect, wall)
objective = Or(region, Not("obstacles"))
# out = objective.realize(impl, obstacles=...)

# out = TLT.construct(env).realize(impl)
# plot2D_levelset(impl.project_onto(out, 0, 1), 
#                 min_bounds=min_bounds[:2],
#                 max_bounds=max_bounds[:2])

sb1=<ReferredSet at 0x000007f5be00f6f6>: ()
sb2=<ReferredSet at 0x000007f5be00f6f0>: ()
x=<AppliedSet at 0x000007f5c2d4c561>: ('intersect',)
sb1=<ReferredSet at 0x000007f5bdbe6546>: ()
sb2=<ReferredSet at 0x000007f5bdbe6528>: ()
x=<AppliedSet at 0x000007f5be01be1e>: ('intersect',)
